In [20]:
import pandas as pd
import json

In [21]:
file_path = "C:\\Users\\USER\\Downloads\\yelp_dataset\\yelp_academic_dataset_review.json"

In [22]:
# Read the json file in the given file path
reviews = pd.read_json(file_path, lines=True, chunksize=10000)

In [25]:
import pandas as pd

def process_data(reviews_reader, max_rows=10000, min_text_length=100, max_text_length=128):
    processed_df = None
    while True:
        df = next(reviews_reader)
        
        # Filter rows based on text length and select columns 'text' and 'stars'.
        df = df.loc[(df['text'].str.len() > min_text_length) & (df['text'].str.len() <= max_text_length), ['text', 'stars']]
        
        if processed_df is None:
            processed_df = df
        else:
            # Concatenate the unclean DataFrame with the processed DataFrame.
            processed_df = pd.concat([processed_df, df], ignore_index=True)
        
        for rating in range(1, 6):
            rating_df = processed_df[processed_df['stars'] == rating]
            if len(rating_df) > max_rows // 5:
                # Select the top nrows // 5 rows for a specific rating.
                rating_df = rating_df.iloc[:max_rows // 5]
                
                # Remove rows with the same rating from the processed DataFrame.
                processed_df = processed_df.drop(processed_df[processed_df['stars'] == rating].index)
                
                # Concatenate the filtered rating DataFrame with the processed DataFrame.
                processed_df = pd.concat([processed_df, rating_df], ignore_index=True)
        
        if len(processed_df) >= max_rows:
            return processed_df


In [26]:
train_df = process_data(reviews)
test_df = process_data(reviews)

In [27]:
train_df.head()
test_df.head()

,text,stars
0,Retail stores closing early ( over an hour) is...,1
1,Didn't tell me how much the labor will cost an...,1
2,I'll just say I have had better! I won't be go...,1
3,This place is horrible. The food is so salty t...,1
4,Wow. I ordered soup because I'm getting sick a...,1
